In [7]:
import helper
import stream
import importlib
import pickle
import os
import sys
from math import ceil, log2

# Test helper

In [17]:
l = [
    {
        "a": [1, 2, 3],
        "b": [2, 4, 5]
    },
    {
        "a": [6, 7],
        "b": [6, 8, 9, 10],
        "c": [9]
    },
    {
        "a": [11, 12, 13],
        "b": [11, 14],
        "c": [13]
    }, 
    {
        "a": [15, 16, 17],
        "b": [18],
        "c": [15, 18, 19]
    }
]

with open("stoplist.txt", encoding="latin1") as file:
    stoplist = [line.rstrip().lower() for line in file]

In [4]:
for i, d in enumerate(l):
    filename = f"f{i}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(d, file)

In [61]:
importlib.reload(helper)
h = helper.Helper(0, 4)
print(h.files)
print(h.invidx)
print(h.word)
print(h.idoc)
print(h.docID)
print(h.i)

['f0.pkl', 'f1.pkl', 'f2.pkl', 'f3.pkl']
[('a', [1, 2, 3]), ('b', [2, 4, 5])]
a
0
1
0


# Test stream

In [26]:
dicts = []
with open("stream.pkl", "rb") as f:
    print(type(f))
    for _ in range(ss.list_count):
        dict = pickle.load(f)
        dicts.append(dict)

<class '_io.BufferedReader'>


In [40]:
importlib.reload(stream)
files = all_files_and_dirs = os.listdir("lyrics/")
files = ["lyrics/" + file for file in files]
ss = stream.Stream(files, stoplist)

Document lyrics/505.txt has been preprocessed!
Document lyrics/arabella.txt has been preprocessed!
Document lyrics/cornerstone.txt has been preprocessed!
Document lyrics/dance_little_liar.txt has been preprocessed!
Document lyrics/no1_party_anthem.txt has been preprocessed!


In [66]:
ss.next()

('d', 'lyrics/505.txt')

In [67]:
ss.close_file()

In [129]:
importlib.reload(stream)

files = all_files_and_dirs = os.listdir("minitexts/")
files = ["minitexts/" + file for file in files]

ss2 = stream.Stream(files, stoplist)

Document minitexts/t1.txt has been preprocessed!
Document minitexts/t2.txt has been preprocessed!
Document minitexts/t3.txt has been preprocessed!


In [136]:
ss2.next()

('re', 'minitexts/t3.txt')

In [92]:
dicts = []
with open("stream.pkl", "rb") as f:
    for _ in range(ss2.list_count):
        dict = pickle.load(f)
        dicts.append(dict)

<class '_io.BufferedReader'>


In [120]:
ss.close_file()
ss2.close_file()

In [125]:
def test(s):
    print(s.next())
test(ss2)

('sid', 'minitexts/t2.txt')


In [126]:
ss2.close_file()

In [128]:
import sys
dict = {
    "hello":2
}
sys.getsizeof(dict)

184

# Test invert index

In [16]:
directory = "D:/Documents-D/Testing/spimi_files_to_merge"
with open("f0.pkl", "rb") as f:
    dict = pickle.load(f)
dict

{'hell': [('minitexts/t1.txt', 3), ('minitexts/t2.txt', 1)],
 'wondering': [('minitexts/t1.txt', 1), ('minitexts/t3.txt', 1)],
 'sid': [('minitexts/t2.txt', 1)],
 're': [('minitexts/t3.txt', 1)],
 'prowl': [('minitexts/t3.txt', 1)],
 'wheth': [('minitexts/t3.txt', 1)],
 'left': [('minitexts/t3.txt', 1)],
 'already': [('minitexts/t3.txt', 1)]}

# Test logic :skull:

In [29]:
n = 4
MAX_DICT_SIZE = 1000
for i in range(ceil(log2(n))):
    last_file = 0 # index of last file
    for j in range(ceil(n / pow(2, i + 1))):
        group_size = pow(2, i)
        h1 = helper.Helper(last_file, last_file + group_size)
        last_file += group_size
        h2 = helper.Helper(last_file, last_file + group_size)
        last_file += group_size

        # listas de resultados (cambiar para que sólo haya 1 simultáneamente)
        l1 = {}
        l2 = {}
        l = l1

        while not h1.over and not h2.over:
            if h1.word < h2.word or (h1.word == h2.word and h1.docID < h2.docID):
                if h1.word not in l:
                    l[h1.word] = [h1.get_next_pair()]
                else:
                    l[h1.word].append(h1.get_next_pair())
            elif h2.word < h1.word or (h1.word == h2.word and h2.docID <= h1.docID):
                if h2.word not in l:
                    l[h2.word] = [h2.get_next_pair()]
                else:
                    l[h2.word].append(h2.get_next_pair())     
            if sys.getsizeof(l) > MAX_DICT_SIZE:
                l = l2
        while not h1.over:
            if h1.word not in l:
                l[h1.word] = [h1.get_next_pair()]
            else:
                l[h1.word].append(h1.get_next_pair())
        while not h2.over:
            if h2.word not in l:
                l[h2.word] = [h2.get_next_pair()]
            else:
                l[h2.word].append(h2.get_next_pair())
        

[[2], [4, 5, 5, 7, 9], [5], [5, 6, 7, 7, 9], [2], [4, 5, 5, 6, 7]]

In [ ]:
n = 4
MAX_DICT_SIZE = 1000
for i in range(ceil(log2(n))):
    last_file = 0 # index of last file
    for j in range(ceil(n / pow(2, i + 1))):
        group_size = pow(2, i)
        wbuffer = helper.WriteBuffer(last_file, last_file + 2 * group_size)
        h1 = helper.Helper(last_file, last_file + group_size)
        last_file += group_size
        h2 = helper.Helper(last_file, last_file + group_size)
        last_file += group_size

        while not h1.over and not h2.over:
            h = h1
            if h2.word < h1.word or (h1.word == h2.word and h2.docID <= h1.docID):
                h = h2
            if not wbuffer.insert(h.get_next_pair()):
                print("ERROR AT " + str(i) + ", " + str(j))

        while not h1.over:
            if not wbuffer.insert(h1.get_next_pair()):
                print("ERROR AT " + str(i) + ", " + str(j))

        while not h2.over:
            if not wbuffer.insert(h2.get_next_pair()):
                print("ERROR AT " + str(i) + ", " + str(j))

# Test all

In [131]:
MAX_DICT_SIZE = 1000
dir1 = "dir1/"
dir2 = "dir2/"
importlib.reload(stream)
import invert_index
importlib.reload(invert_index)
importlib.reload(helper)

with open("stoplist.txt") as file:
    stoplist = [line.rstrip().lower() for line in file]

files = os.listdir("lyrics/")
files = ["lyrics/" + file for file in files]
ss = stream.Stream(files, stoplist)

# SPIMIndexConstruction!
files_to_merge = []
n = 0
while not ss.empty:
    fn = invert_index.spimi_invert(dir2, ss, n)
    files_to_merge.append(fn)
    n += 1
print(files_to_merge)

Document lyrics/4_out_of_5.txt has been preprocessed!
Document lyrics/505.txt has been preprocessed!
Document lyrics/arabella.txt has been preprocessed!
Document lyrics/batphone.txt has been preprocessed!
Document lyrics/cornerstone.txt has been preprocessed!
Document lyrics/dance_little_liar.txt has been preprocessed!
Document lyrics/no1_party_anthem.txt has been preprocessed!
Document lyrics/one_for_the_road.txt has been preprocessed!
Document lyrics/when_the_sun_goes_down.txt has been preprocessed!
['dir2/f0.pkl', 'dir2/f1.pkl', 'dir2/f2.pkl', 'dir2/f3.pkl', 'dir2/f4.pkl', 'dir2/f5.pkl', 'dir2/f6.pkl', 'dir2/f7.pkl', 'dir2/f8.pkl', 'dir2/f9.pkl', 'dir2/f10.pkl', 'dir2/f11.pkl', 'dir2/f12.pkl', 'dir2/f13.pkl', 'dir2/f14.pkl', 'dir2/f15.pkl', 'dir2/f16.pkl', 'dir2/f17.pkl', 'dir2/f18.pkl', 'dir2/f19.pkl', 'dir2/f20.pkl', 'dir2/f21.pkl', 'dir2/f22.pkl', 'dir2/f23.pkl', 'dir2/f24.pkl']


In [57]:
# Merge
importlib.reload(helper)

for i in range(ceil(log2(n)) ):
    last_file = 0 # index of last file
    for j in range(ceil(n / pow(2, i + 1))):
        # group_size = pow(2, i)
        # wbuffer = helper.WriteBuffer(last_file, last_file + 2 * group_size)
        # h1 = helper.Helper(last_file, last_file + group_size, n)
        # last_file += group_size
        # h2 = helper.Helper(last_file, last_file + group_size, n)
        # last_file += group_size
        group_size = pow(2, i)
        wbuffer = helper.WriteBuffer(last_file, last_file + 2 * group_size)
        h1 = helper.Helper(last_file, last_file + 2*group_size, n, 0)
        h2 = helper.Helper(last_file, last_file + 2*group_size, n, 1)
        last_file += 2*group_size

        while not h1.over and not h2.over:
            h = h1
            if h2.word < h1.word or (h1.word == h2.word and h2.docID <= h1.docID):
                h = h2
            elem = h.get_next_pair()
            if i == ceil(log2(n)) -1:
                print(elem)
            wbuffer.insert(elem[0], elem[1])
            # if not wbuffer.insert(elem[0], elem[1]):
                # print("ERROR AT " + str(i) + ", " + str(j))

        while not h1.over:
            elem = h1.get_next_pair()
            if i == ceil(log2(n)) -1:
                print(elem)
            wbuffer.insert(elem[0], elem[1])
            # if not wbuffer.insert(elem[0], elem[1]):
            #     print("ERROR AT " + str(i) + ", " + str(j))

        while not h2.over:
            elem = h2.get_next_pair()
            if i == ceil(log2(n)) -1:
                print(elem)
            wbuffer.insert(elem[0], elem[1])
            # if not wbuffer.insert(elem[0], elem[1]):
            #     print("ERROR AT " + str(i) + ", " + str(j))

        wbuffer.end_round()


WRITE BUFFER FILES:  ['f0.pkl', 'f1.pkl']
0
read file  f0.pkl
1
read file  f1.pkl
--> wrote  f0.pkl
--> wrote  f1.pkl
WRITE BUFFER FILES:  ['f2.pkl', 'f3.pkl']
0
read file  f2.pkl
1
read file  f3.pkl
--> wrote  f2.pkl
--> wrote  f3.pkl
WRITE BUFFER FILES:  ['f4.pkl', 'f5.pkl']
0
read file  f4.pkl
1
read file  f5.pkl
--> wrote  f4.pkl
--> wrote  f5.pkl
WRITE BUFFER FILES:  ['f6.pkl', 'f7.pkl']
0
read file  f6.pkl
1
read file  f7.pkl
--> wrote  f6.pkl
--> wrote  f7.pkl
WRITE BUFFER FILES:  ['f8.pkl', 'f9.pkl']
0
read file  f8.pkl
1
--> wrote  f8.pkl
--> wrote  f9.pkl
WRITE BUFFER FILES:  ['f0.pkl', 'f1.pkl', 'f2.pkl', 'f3.pkl']
0
read file  f0.pkl
1
read file  f1.pkl
read file  f2.pkl
--> wrote  f0.pkl
--> wrote  f1.pkl
read file  f3.pkl
--> wrote  f2.pkl
--> wrote  f3.pkl
WRITE BUFFER FILES:  ['f4.pkl', 'f5.pkl', 'f6.pkl', 'f7.pkl']
0
read file  f4.pkl
1
read file  f5.pkl
read file  f6.pkl
--> wrote  f4.pkl
--> wrote  f5.pkl
read file  f7.pkl
--> wrote  f6.pkl
--> wrote  f7.pkl
WRITE BU

In [132]:
importlib.reload(helper)

for i in range(ceil(log2(n)) ):
    last_file = 0 # index of last file
    dir1, dir2 = dir2, dir1
    for j in range(ceil(n / pow(2, i + 1))):
        group_size = pow(2, i)
        wbuffer = helper.WriteBuffer(dir2, last_file, last_file + 2 * group_size, n)
        h1 = helper.Helper(dir1, last_file, last_file + group_size, n)
        last_file += group_size
        h2 = helper.Helper(dir1, last_file, last_file + group_size, n)
        last_file += group_size

        while not h1.over and not h2.over:
            h = h1
            if h2.word < h1.word or (h1.word == h2.word and h2.docID <= h1.docID):
                h = h2
            elem = h.get_next_pair()
            if i == ceil(log2(n)) -1:
                print(elem)
            wbuffer.insert(elem[0], elem[1])
            # if not wbuffer.insert(elem[0], elem[1]):
                # print("ERROR AT " + str(i) + ", " + str(j))

        while not h1.over:
            elem = h1.get_next_pair()
            if i == ceil(log2(n)) -1:
                print(elem)
            wbuffer.insert(elem[0], elem[1])
            # if not wbuffer.insert(elem[0], elem[1]):
            #     print("ERROR AT " + str(i) + ", " + str(j))

        while not h2.over:
            elem = h2.get_next_pair()
            if i == ceil(log2(n)) -1:
                print(elem)
            wbuffer.insert(elem[0], elem[1])
            # if not wbuffer.insert(elem[0], elem[1]):
            #     print("ERROR AT " + str(i) + ", " + str(j))

        wbuffer.end_round()

WRITE BUFFER FILES:  ['dir1/f0.pkl', 'dir1/f1.pkl']
read file  dir2/f0.pkl
read file  dir2/f1.pkl
--> wrote  dir1/f0.pkl
--> wrote  dir1/f1.pkl
WRITE BUFFER FILES:  ['dir1/f2.pkl', 'dir1/f3.pkl']
read file  dir2/f2.pkl
read file  dir2/f3.pkl
--> wrote  dir1/f2.pkl
--> wrote  dir1/f3.pkl
WRITE BUFFER FILES:  ['dir1/f4.pkl', 'dir1/f5.pkl']
read file  dir2/f4.pkl
read file  dir2/f5.pkl
--> wrote  dir1/f4.pkl
--> wrote  dir1/f5.pkl
WRITE BUFFER FILES:  ['dir1/f6.pkl', 'dir1/f7.pkl']
read file  dir2/f6.pkl
read file  dir2/f7.pkl
--> wrote  dir1/f6.pkl
--> wrote  dir1/f7.pkl
WRITE BUFFER FILES:  ['dir1/f8.pkl', 'dir1/f9.pkl']
read file  dir2/f8.pkl
read file  dir2/f9.pkl
--> wrote  dir1/f8.pkl
--> wrote  dir1/f9.pkl
WRITE BUFFER FILES:  ['dir1/f10.pkl', 'dir1/f11.pkl']
read file  dir2/f10.pkl
read file  dir2/f11.pkl
--> wrote  dir1/f10.pkl
--> wrote  dir1/f11.pkl
WRITE BUFFER FILES:  ['dir1/f12.pkl', 'dir1/f13.pkl']
read file  dir2/f12.pkl
read file  dir2/f13.pkl
--> wrote  dir1/f12.pkl
--> 

In [42]:
with open("f7.pkl", "rb") as f:
    dict = pickle.load(f)
dict

{'need': [('lyrics/dance_little_liar.txt', 1)],
 'nev': [('lyrics/dance_little_liar.txt', 2)],
 'newly': [('lyrics/dance_little_liar.txt', 1)],
 'normal': [('lyrics/dance_little_liar.txt', 1)],
 'parrot': [('lyrics/cornerstone.txt', 1)],
 'phon': [('lyrics/cornerstone.txt', 1)],
 'plac': [('lyrics/cornerstone.txt', 1)],
 'really': [('lyrics/cornerstone.txt', 1)],
 'spotless': [('lyrics/dance_little_liar.txt', 1)],
 'wrong': [('lyrics/dance_little_liar.txt', 1)]}

# Test meths

In [48]:
n = 7
for i in range(ceil(log2(n))):
    last_file = 0 # index of last file
    for j in range(ceil(n / pow(2, i + 1))):
        print("i:", i, ", j: ", j)
        group_size = pow(2, i)
        print("first file ", last_file, ": ", last_file)
        last_file += 2 * group_size
        print("last file ", last_file, ": ", last_file)
        print("------------------------")

i: 0 , j:  0
first file  0 :  0
last file  2 :  2
------------------------
i: 0 , j:  1
first file  2 :  2
last file  4 :  4
------------------------
i: 0 , j:  2
first file  4 :  4
last file  6 :  6
------------------------
i: 0 , j:  3
first file  6 :  6
last file  8 :  8
------------------------
i: 1 , j:  0
first file  0 :  0
last file  4 :  4
------------------------
i: 1 , j:  1
first file  4 :  4
last file  8 :  8
------------------------
i: 2 , j:  0
first file  0 :  0
last file  8 :  8
------------------------
